# <img style="float: right;"  src="images/jp.png" width="200">

# Module mmVars

This document describes the [mmVars module](http://localhost:8888/edit/Code/mmVars.py)

Version 1.1 (12/4/2019)  
License information is at the end of the document

---

The module defines random variables bounded by **minimum** and **maximum** values. Using those variables you can find the minimum and maximum of some functions that use those variables.

For some functions, the minimum and maximum is not so easy to find from the limits of its variables. In those cases this module provides bounds to the maximum and mimimum values.

For any generic function that uses the random variables the module also provides the **montecarlo** method that obtains the distribution of the function values from which a rough estimate of maximum and minimum values can also be obtained.

You can find [this module](http://localhost:8888/edit/Code/mmVars.py) on the [Code folder](http://localhost:8888/tree/Code)

## Importing the module

The following code imports the **mmVars** module.

It also loads and imports the **calc** module that is used for graphics and the **numpy** module.

In [ ]:
# Import all needed modules
import numpy as np  
import mmVars as mm
import calc

# Check loaded modules
try:
    print('mmVars version: ',mm.version)
    print('calc version: ',calc.version)
except:
    print('Error loading modules')
    raise

## mmVar Class

The **mmVar** class is the main element on the mmVars module.

It defines an object associated to **minimum** and **maximum** values.

You can define a new mmVar object using different constructors. Assuming that the module is imported as **mm**, the generic constructor is:

>`var = mm.mmVar(a,b=None,typ=None,tol=None,s=None,ns=0)`

The constructor is complex because it admits several modes.

For instance, you can use the following code to define a variable with values between **min** and **max**:

>`var = mm.mmVar(min,max)`

The module automatically detects wich is maximum and minimum, so the following code is equivalent:

>`var = mm.mmVar(max,min)`

You can also define a **typical** value for the variable that must be between the maximum and minimum values:

>`var = mm.mmVar(max,min,typ)`

The variable can also be defined by a **central value** and a **tolerance**:

>`var = mm.mmVar(central,tol)`

In this case it defines:

>$max = central \cdot (1+tol)$

>$min = central \cdot (1-tol)$

>$typ = central$

If you only provide one parameter, it will define a constant:

>`var = mm.mmVar(value)`

So that:

>$max = min = typ = value$

We can define two possible **probability distributions** to be used in later **montecarlo** methods. By default the distribution is [uniform](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)), that means that all possible values between the maximum and minimum are equally probable.

You can define a [normal](https://en.wikipedia.org/wiki/Normal_distribution) distribtion if, in any of the above constructor methods, a parameter **ns** is used, like in:

>`var = mm.mmVar(min,max,ns=3)`

The **ns** paramenter defines the number of [standard deviations](https://en.wikipedia.org/wiki/Standard_deviation) $\sigma$ between the central and the maximum or minimum values. For instance, ns value of 3 means that any value of the variable has a 99,7% chance to be in the region between the defined maximum and the minimum values. Note that a normal distribution has **no bounds**, so random values can be outside of the region defined by the maximum and minimum values.

If you don't define a **typical** value for a variable defined with a **normal distribution**, the typical value will be set equal to the center value:

>$typ_{normal}=\frac{max+min}{2}$

In **uniform** distribution variables the typical value can be left **undefined**.

In the case of **normal** distributions you can also define the $\sigma$ value using the **s** parameter of the constructor. That way, the following constructor:

>`var = mmVars.(10,s=1)`

Will define a **normal** variable with $\sigma=1$. By default the limits are defined at $3\sigma$ so the **maximum** of the variable will be 13 and the **minimum** will be 7. If you want, you can define the $\sigma$ and the **ns** at the same time, so:

>`var = mmVars.(10,s=1,ns=2)`

Will define a **normal** variable with $\sigma=1$, **maximum** will be 12 and **minimum** will be 8 because we have defined they to be at a $2\sigma$ distance from the certer value.


Let's see some **constructor** examples. The limits of each variable is shown as **(max:typ:min)**

As the code cell is interactive you can try to add your own examples:

In [ ]:
# Helper function for the examples
# You are not supposed to directly access the ns value although it works
def showVar(text,x):
    if x.ns == 0:
        print(text,' ',x,' Uniform')
    else:    
        print(text,' ',x,' Normal ns =',x.ns
              ,', s =',(x.maximum()-x.minimum())/(2*x.ns))

# Constructor examples
print('Uniform variables')
showVar('mmVar(1,3)',mm.mmVar(1,3))
showVar('mmVar(1,3,2)',mm.mmVar(1,3,2))
showVar('mmVar(10,tol=0.1)',mm.mmVar(10,tol=0.1))
print()
print('Normal variables')
showVar('mmVar(1,3,ns=3)',mm.mmVar(1,3,ns=3))
showVar('mmVar(10,s=1)',mm.mmVar(10,s=1))
showVar('mmVar(10,s=1,ns=2)',mm.mmVar(10,s=1,ns=2))
showVar('mmVar(10,tol=0.1,s=1)',mm.mmVar(10,tol=0.1,s=1))
print()
print('You should not define the limits, s and ns')
print('If you give too much information, ns takes precedence over s')
showVar('mmVar(10,tol=0.1,s=1,ns=2)',mm.mmVar(10,tol=0.1,s=1,ns=2))

Let's see two examples of generating variables and the code that shows the distributions. Don't worry about the **doMontecarlo** , **histogram** and **generic** functions for now, they will be described later.

In [ ]:
# Define an uniform variable between 2 and 10
a = mm.mmVar(2,10)

# Calculate and draw the histogram
vRes,vData = mm.doMontecarlo(10000,lambda x:x,a)
calc.plotHist(vRes,50,"'a' variable histogram","Values","Frequency")
a.generic()

In [ ]:
# Define a norma variable with central value 10 and tolerance of 10% (3 sigmas)
b = mm.mmVar(10,tol=0.1,ns=3)

# Calculate and draw the histogram
vRes,vData = mm.doMontecarlo(10000,lambda x:x,b)
calc.plotHist(vRes,50,"'b' variable histogram","Values","Frequency")
b.generic()

Observe that, there are some cases that are outside of the $3\sigma$ range that is, in this case, between 9 and 11.

## Getting information about a mmVar object

You can get information about a variable defined as **mmVar** object using the following three methods:

>* var.maximum()

>* var.minimum()

>* var.typical()

There is no member function to obtain the **ns** or the $\sigma$ values. As you have created the variables, you already need to know. You could say the same about **maximum**, **minimum** and **typical** values, but those values, as we will see later, will be propagated when operating with **mmVar** objects whereas **normal** parameters are discarded.

If you **print** a **mmVar** object you will use the **str** method that generates a strings with the following format:

>`(max:typ:min)`

The code below shows how to access to the variable contents

In [ ]:
# Variable 'a' information

print("Printing a: ",a)
print("a maximum: ",a.maximum())
print("a minimum: ",a.minimum())
print("a typical: ",a.typical())

print()
  
# Variable 'b' information

print("Printing b: ",b)
print("b maximum: ",b.maximum())
print("b minimum: ",b.minimum())
print("b typical: ",b.typical())


## Freeze to typical

You can freeze the value of a **mmVar** object to its typical value using the **setTypical()** method. If the typical value is **undefined** you will get an exception.

>`var.setTypical()`

Once you have **frozen** the variable, it will behave as a constant value. The above method also returns the typical frozen value of the variable.

If you want to revert to the normal **unfrozen** variable you can use the **generic()** method:

>`var.generic()`

There are two mmVars module functions that ease the freeze and unfreeze of several variables at the same time. Assuming you imported the module as **mm** you can set severa variables to the typical value using the **setTypical** function:

>`mm.setTypical(var1,var,var3,...)`

This function also returns a tuple with the typical values of the variables

If you want to return several variables to the default state you can use the **generic** function. This one don't return anything

>`mm.generic(var1,var,var3,...)`

The following code shows the frozen variable operation

In [ ]:
print('Current variable b: ',b)

value = b.setTypical()  
print('Freezing b to typical value ',value)

print('Current variable b: ',b)

print('Unfreezing to generic')
b.generic()

print('Current variable b: ',b)

print()
print("Trying to set typical on 'a' generates an exception")
try:
    a.setTypical()
except:
    pass
  
print("New variable 'c' uniform with a non centered typical value")
c = mm.mmVar(-2,7,5)
print('Variable c: ',c)

print()
print('Freeze to typical b and c variables at once')
t=mm.setTypical(b,c)

print('Current b: ',b,' Current c: ',c)
print('Tuple returned: ',t)

print()
print('Set to generic both variables at once')
mm.generic(b,c)
print('Current b: ',b,' Current c: ',c)


## Montecarlo : Freeze to random

You can also freeze one variable to have a **random** value between the **maximum** and the **minimum** one. The random value will have the selected distribution for the variable **uniform** or **normal**. 

Use the **montecarlo** method to freeze the variable to a random value. The [montecarlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) name comes from gambling and is also the name given to the use of random numbers to solve, for instance, optimization problems.

>`var.montecarlo()`

As in the **setTypical** method, this method returns the random value set on the variable.

Remember that the **normal** distribuion is really **unbound** so values outside the maximum to minimum range can be generated if you use this distribution.

Once you have frozen the variable, it will behave as a constant value.  

The following code shows the **montecarlo** method

In [ ]:
a.generic() # Added for re-runs of this cell
print("Uniform variable a: ",a)

a.montecarlo()
print("Now it's frezed to value ",a)
print("Still the same value ",a)

print()
print('New montecarlo calls generate new values')
print()
for i in range(10):
    print("    Montecarlo instance #",i+1," : ",a.montecarlo())



## Optimization with doMontecarlo

The montecarlo method is usual in **optimization** problems.

Let's say you have a function that depends on several variables and you want to know the **range of values** the function can generate. One way to solve the problem is to give as input values to the function random values inside their valid ranges. Each time the funcion is evaluated with different random values is called a **run**

The function **doMontecarlo** perfoms **n** runs:

>`vRes,vData = doMontecarlo(n,func,*vars)`

Where:

>**func** is a function that depends on several variables

>**n** is the number of evaluations of the function with different random values

>**\*vars** is the list of variables (can be **mmVar** objects) that are arguments of the function **func**

The function returns two sorted lists:

>**vRes** is a list of the n values obtained evaluating the function **func**

>**vData** is a list of n elements, one for each montecarlo run, that are also lists of two elements:

>>The value obtained evaluating **func** (same as in vRes)

>>A tuple with the **func** arguments for this value

As indicated the two returned lists are ordered. The first element corresponds to the run that has given the lowest return value from **func** and the last element is the one that has given the highest return value.

Using the **vData** return list you can not only see the maximum and minimum values of the set of runs, but also you can see the arguments used on **func** for those values.


##Montecarlo Histogram

One way to represent the information returned by the **doMontecarlo** function is to plot an [histogram](https://en.wikipedia.org/wiki/Histogram) of the **vRes** vector.

In order to draw an histogram from a vector of values, we divide the range of values contained in the vector in a **$n_{Bins}$** number of bins. For instance, if the range of values on the vector is between 0 an 10 and we use 10 bins, the first bin will include values between 0 and 1, the second between 1 and 2, and son on...

Then we include each value in the vector to the bin where it belongs. Afterwars we represent the bins in a **bar chart** using, for each bar, a height equal to the number of elements in the bin.

The **plotHist** function in the **calc** module plots a vector histogram using the **matplotlib** module.

>>`calc.plotHist(vRes,n,Title,x_label,y_label)`

Where:

>**vRes** is the vector to represent as an histogram

>**n** is the number of bins

>**Title** is the plot tittle string

>**x_label** is the label string on x axis

>**y_label** is the label string on y axis

Everything is easier with an **example**, so here it is:

## Optimization example

You have a series connection of a supply **$V_{DD}$** a resistor **$R$** and a LED diode with forward voltage **$V_D$**. The current on the diode will be:

>$I_{LED}=\frac{V_{DD}-V_D}{R}$

Suppose that the supply $V_{DD}$ can have any uniform value between 4.5V and 5V,the voltage on the LED $V_D$ is normal centered arround 1.5V with $3\sigma$ up to 1.4V and 1.6V and the resistor is $470\Omega$ nominal with a 5% tolerance defined as $2\sigma$

The following code shows, in an **histogram**, the expected distribution range of the current on the LED

The code also shows, using the **vData** vector, the **maximum** and **minimum** values of the LED current together with the values of $V_{DD}$, $V_D$ and $R$ related to them.

In [ ]:
# Random variables
vdd  = mm.mmVar(4.5,5)
vled = mm.mmVar(1.4,1.6,ns=3)
R    = mm.mmVar(470,tol=0.05,ns=3)

# Current as function of parameters
def current(vsup,vd,r):
    return 1000*(vsup-vd)/r  # In mA

# Montecarlo simulation with 10000 runs
vRes,vData = mm.doMontecarlo(10000,current,vdd,vled,R)

# Current histogram
calc.plotHist(vRes,50,"LED current histogram","Curent (mA)","Frequency")

# Show limits
print()
print('Maximum current is ',vRes[-1]," mA when")
print('\tVdd is ',vData[-1][1][0],' V')
print('\tVled is ',vData[-1][1][1],' V')
print('\tR is ',vData[-1][1][2],' Ohm')
print()
print('Minimum current is ',vRes[0]," mA when")
print('\tVdd is ',vData[0][1][0],' V')
print('\tVled is ',vData[0][1][1],' V')
print('\tR is ',vData[0][1][2],' Ohm')
print()

This is simple case, you could expect that the maximum current will take place when the supply is **5V**, the led voltage is **1.4V** and the resistor is minimum at a 5% below $470\Omega$ (that equals $446.5\Omega$). Maximum LED current will be, in this case:

>$I_{LED\:max}=\frac{V_{DD\:max}-V_{D\:min}}{R_{min}}=\frac{5V-1.4V}{446.5}=8.06\:mA$


You can see that the obtained results are not too far from the expected values. The solution not only shows the **maximum** and **minimum** of the current but also the probability **distribution** using an **histogram**.

## Cumulative function

By integrating the information on the **vRes** vector we can obtain the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) of the LED current. The **cumulative** function of the **mmVars** module gives, for each value of the input vector **v**, the number of vector elements that smaller or equal than this value divided by the total number of elements in the vector.

>$f_{Cumulative}(x)=\frac{elements \: less \: than \: x}{total \: number \: of \: elements}$

The **cumulative** function takes a **sorted v1** vector and returns an equal sized **v2** vector

>`v2 = cumulative(v1)`

As input vector is sorted, vector **v2** is just a sequence:

>$\frac{1}{n} \quad \frac{2}{n} \quad \frac{3}{n} \quad \frac{4}{n} \quad ... \quad \frac{n}{n} $

The following **code** shows the cumulative function associated to the **LED current** in the example

In [ ]:
# Cumulative funcion
cf = mm.cumulative(vRes)
calc.plot11(vRes,cf,"Cumulative function","Current (mA)","Probability")

## Probability for a range of values

Given a **cumulative function**, the probability of having a value between $X_{min}$ and $X_{max}$ is:

>$prob=f_{Cumulative}(X_{max})-f_{Cumulative}(X_{min})$

The **prob** function of the **mmVars** module just does that:

>`prob(vRes,a,b)`

It returns the probability that a random value from the vector **vRes** is between **a** and **b**. The **prob** function gets the maximum and minimum from the a and b vales so that *prob(vRes,a,b)* and *prob(vRes,b,a)* give the same result.

The following **code** use the **prob** function to calculate the probability to have a LED current between two given values.

In [ ]:
# Prob
p = mm.prob(vRes,6.25,7.5)
print('Probability of LED current to be between 6.25 mA and 7.5 mA is ',100*p,'%')

## Operating with mmVar objects

Using the **montecarlo** methods you can obtain an approximation of the **maximum** and **minimum** values of a function that has parameters defined as **mmVar** objects. This, however, does not guarantee obtaining the **real** maximum and minimum values. In the previous LED example, for instance, the montecarlo method does not give exactly the expected 8.06 mA value.

You can operate with **mmVar** objects using the standard **+**, **-**, ***** and **/** operators as seen on the following **example**

In [ ]:
# Two mmVar objects
a = mm.mmVar(1,5,2)
b = mm.mmVar(10,20,15)
print('a = ',a)
print('b = ',b)

# Some operations
print()
print('a + b = ',a+b)
print('a - b = ',a-b)
print('a * b = ',a*b)
print('a / b = ',a/b)
print('-a = ',-a)

When you generate a new **mmVar** object by making some operations with other **mVar** objects, you get the proper **maximum** and **minimum** limits, but new objects will always get the default **uniform** distribution. 

That means that normal distribution information is lost when operating on **mmVar** objects. Note that, after operating on objects, the distribution of the result **won't be really uniform**. In fact, due to the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem), adding several random variables tend to yield a normal distribution regardless of the distribution of the original variables. The problem is more complex when doing other operations different of just adding. The **mmVars** module is not capable to deduce how the **probability function** changes after operating on **mmVar** objects.

You can also operate mixing **mmVar** objects and constat values:

In [ ]:
print('a = ',a)
print('b = ',b)
print()
print('2 * a = ',2*a)
print('10 / b = ',10/b)
print('a + 4 = ',a+4)

Trying to divide by a **mmVar** object that includes **zero** in the range will generate an exception as it includes $\infty$ in the range.

In [ ]:
c = mm.mmVar(1,-1,0)

print('c = ',c)

print("Tryin to execute print('1/c = ',1/c)")

try:
    print('1/c = ',1/c)
except:
    pass

The **mmVar** module also includes the following functions:

>|||
>|---|---|
>|**sq(x)** &nbsp; &nbsp; | Square of a value (Don't work with negatives)|
>|**sqrt(x)** &nbsp; &nbsp; | Square root of a value|
>|**exp(x)** &nbsp; &nbsp; | Exponential function|
>|**log(x)** &nbsp; &nbsp; | Natural logarithm (Don't work with negatives)|
>|**ipow(x,e)** &nbsp; &nbsp; | x^e with constant positive integer e|
>|**sin(x)** &nbsp; &nbsp; | Sine function|
>|**cos(x)** &nbsp; &nbsp; | Cosine function|

Functions **sqrt(x)** and **log(x)** only work on positive numbers (zero is also not valid for **log(x)**). If you try to use them on negative numbers, an exception will be generated.

If you use any of the above functions with float or integer values you will get the normal expected functions. If you use them with **mmVar** objects, you will get the proper **max** and **min** limits. The **typical** value will be calculated just making the operation on the typical value. As commented before, any information on the **probability function** will be lost.

The following **example** shows some cases:

In [ ]:
d = mm.mmVar(2,-3,-1)

print('a = ',a)
print('b = ',b)
print('c = ',c)
print('d = ',d)

print()

print('sq(d) = ',mm.sq(d))
print('sqrt(b) = ',mm.sqrt(b))
print('exp(a) = ',mm.exp(a))
print('log(b) = ',mm.log(b))
print('exp(log(b)) = ',mm.exp(mm.log(b))) # This should yield 'b'
print('ipow(d,3) = ',mm.ipow(d,3))
print('sin(a) = ',mm.sin(a))
print('cos(c) = ',mm.cos(c))

print()
print('Exceptions on out of range values')

try:
    x = mm.sqrt(c)
except:
    pass
  
try:
    x = mm.log(c)
except:
    pass  
 
try:
    x = mm.ipow(a,1.5)
except:
    pass

print('Operation on normal numbers:')

print()

print('sq(3) = ',mm.sq(3))
print('sqrt(64) = ',mm.sqrt(64))
print('exp(1) = ',mm.exp(1))
print('log(10) = ',mm.log(10))
print('exp(log(8)) = ',mm.exp(mm.log(8))) # This should yield 8
print('ipow(2,3) = ',mm.ipow(2,3))
print('sin(2) = ',mm.sin(2))
print('cos(0) = ',mm.cos(0))

## Independence and Individuality

There is a problem when operating on **mmVar** objects.

When you operate on two **mmVar** objects, the code assumes independence between the objects. That is, it is assumed that each object can have any value in the range defined by the **minimum** and **maximum** limits.

An example can give some light on the subject:

In [ ]:
a = mm.mmVar(4,-1,2)

print('a = ',a)

print()

print('a*a = ',a*a)
print('sq(a) = ',mm.sq(a))

print()

b = mm.mmVar(6,1,3)

print('b = ',b)
print('b/b = ',b/b)

You know that **a\*a** should be equal to **sq(a)**. You can see that this is true for the **typical** value where you get the expected **2\*2 = 4** value and the **maximum** value where you get the expected **4\*4 = 16 ** value.

You can also see that the **sq()** formula gives the proper **maximum** and **minumum** values whereas **a\*a** gives a wrong **minimum** value.

When calculating a product **a\*b**, the **mmVars** module determines the range of the **a** value and the range of the **b** value and assumes that both variables **a** and **b** are independent of each other. So, having one value on **a** does not affect the value we can get on **b**.

The same procedure happens when calculating **a\*a**. The **mmVars** module consider both instances of **a** as **generic** and **independent** of each other. 

Note that the **maximum** limit on **a\*a** is right. This is because, in this case, for the **maximum**, having or not independence between the variables makes no difference.

The **independence** problem also explains why **b/b** in the example is also completely **wrong** and don't give the expected **1** result.

You can think that **mmVars** should know better. The problem is that when you operate on two variables, the origin of those variables are lost. 

You cannot expect to have problems only when you operate a variable with itself. You can also have problems whenever you use a variable **two or more times** in the same expression:

In [ ]:
a = mm.mmVar(4,-1,2)

print('a = ',a)
print('a / ( a + 2 ) = ',a*a)

print()

# Show the curve
x = np.arange(a.minimum(),a.maximum(),0.01)
y = x / (x + 2)
calc.plot11(x,y,"","x","x / ( x + 2 )")

As you can see from the **curve**, both the **maximum** and the **minumum** calculated values are **wrong**.

This is because in both cases, independence of the **x** on both sides of the fraction yields a maximum or minimum value of the expression that uses diferent limits for **x**.

>$f_{max}=\frac{x_{max}}{x_{min}+2}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $f_{min}=\frac{x_{min}}{x_{max}+2}$

In this particula case, as the function is [monotonically increasing](https://en.wikipedia.org/wiki/Monotonic_function), the maximum and minimum values can be calculated as:

>$f_{max}=\frac{x_{max}}{x_{max}+2}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $f_{min}=\frac{x_{min}}{x_{min}+2}$

For an x range between **4** and **-1** the limits are **0.66** and **-1** just as the curve shows.

There is no easy generic way to solve this problem when the function depends on several variables. If it **existed** a good solution, some complex [maxima and minima problems](https://en.wikipedia.org/wiki/Maxima_and_minima) will be easily solved.

In any case, assuming independence always gives a **bounding limit** for the function results. For instance, in the example, you are guaranteed that the funcion will be bound between the computed **16** and **-4** values although the real limits are more restrictive.

The **mmVar** object **individual** method eases detecting more than use of a **mmVar** object in a expression. It marks the object as individual so that an **exception** is raised when you try to use the object **twice** as you can see in the **example**:

In [ ]:
a = mm.mmVar(4,-1,2)

print('a = ',a)

print()
print('Try to calculate a*a')

a.individual()

try:
    print('a*a = ',a*a)
except:
    pass
  
print('But calculating sq(a) is ok')

a.individual()
print('sq(a) = ',mm.sq(a))

If you have several variables to mark as **individual** you can use the **individual** function for that. Assuming the **mVars** module is loaded as **mm**, you can use:

>`mm.generic(var1,var,var3,...)`

You shold call the **individual** method or function before the use of the variable on every new expression as the **mmVars** module cannont know if two uses of one variable belong to one or more than one expressions. In fact you can always break **one** complex calculation using temporal variables and **several** expression calculations.

As you can see, the **individual** function is one in the collection of functions that determine how a **mmVar** behaves:

>|||
>|---|---|
>|`mm.generic(var1,var,var3,...)`| &nbsp; &nbsp; Makes the vars generic and always independent (default after creating the variable)|
>|`mm.individual(var1,var,var3,...)`| &nbsp; &nbsp; Makes the vars individual so they use cannot be repeated|
>|`mm.typical(var1,var,var3,...)`| &nbsp; &nbsp; Makes the vars equal to their typical values|
>|`mm.montecarlo(var1,var,var3,...)`| &nbsp; &nbsp; Makes the vars equal to a random value inside their range|


## Practical mmVar operation example

As a practical example we wil use the same formula from the **LED current** used above.


In [ ]:
# This code is repeated to ease the interpretation

vdd  = mm.mmVar(4.5,5)
vled = mm.mmVar(1.4,1.6,ns=3)
R    = mm.mmVar(470,tol=0.05,ns=3)

def current(vsup,vd,r):
    return 1000*(vsup-vd)/r  # In mA

# Now the new code ################################

# Guartee only one use of each variable
mm.individual(vdd,vled,R) 

# Calculate of the LED current using the save function as in montecarlo
iled = current(vdd,vled,R)

# Show the LED current mmVar object
print("Iled [mA] = ",iled)

Note that the program calculates the minumum and maximum values of the LED current that are what we expect to be. That is:

>$I_{LED\:max}=\frac{V_{DD\:max}-V_{D\:min}}{R_{min}}=\frac{5V-1.4V}{446.5}=8.06\:mA$

<BR>

>$I_{LED\:min}=\frac{V_{DD\:min}-V_{D\:max}}{R_{max}}=\frac{4.5V-1.6V}{493.5}=5.87\:mA$

Those values are more exact that the ones obtained using the **montecarlo** method. And they also require less computations.

Observe that the method works because no variable is used more than **once** on the calculations. If a variable was used more than once on the expression this method will yield, at best, a **bouning limit** for the real result. For expressions that use the variables more than once, in general, it is best to use other optimization methods like the **montecarlo** one.


## Best procedure for optimization

Suppose that you have an **optimization** problem where you want to know, the range of values of a function that depends on several input parameters bounded by **maximum** and **minimum** values.
 
>$f(x_1,x_2,x_3,...,x_n)$  &nbsp; &nbsp; with &nbsp; &nbsp; $x_{i\:min} \leq x_i \leq x_{i\:max}$


Which is the best method to use?

It depends. 

For functions that **can** be written so that each variable is used only **once**:

>* You can operate the function on **mmVar** objects to obtain the limits of the function. Note that this don't give any information about the **probability** of having each possible function value.

For functions that **cannot** be written so that each variable is used only **once**:

>* You can operate on **mmVar** objects to obtain the a **bound** limit of the function values. The **bound** limit is not guaranteed to be near the real limits, it only guarantees:

>>>$boudn_{min} \leq func_{min}$ &nbsp; &nbsp; and &nbsp; &nbsp; $func_{max} \leq bound_{max}$

>* You can use the **montecarlo** method to obtain a rough estimation of the function output range of values. Note that you cannot guarantee that you get the real limits. In general:

>>>$func_{min} \leq montecarlo_{min}$ &nbsp; &nbsp; and &nbsp; &nbsp; $montecarlo_{max} \leq func_{max}$

>>But you can get quite close using a high enough number of montecarlo runs

For **any** function, you can use the **montecarlo** method to obtain:

>* The **histogram** of the function possible results

>* The **cumulative** curve of the function possible results

>* The **probability** of having the function results between a given **maximum** and **minimum** value

Note that the **montecarlo** method uses information about the **probability** of the function input variables. So, using the proper definitions for the input variables, as **uniform** or **normal**, and the proper **$\sigma$** value in the normal case, you can get a good information of the **probility** of having any function result.


<BR><BR><BR><BR><BR><BR>

## Document information

Copyright © Vicente Jiménez (2018-2019)

This work is licensed under a [Creative Common Attribution-ShareAlike 4.0 International license](http://creativecommons.org/licenses/by-sa/4.0/). 

The **mmVars.py** code is licensed under the [MIT License](https://opensource.org/licenses/MIT).

You can find the module [here](https://github.com/R6500/Python-bits/tree/master/Modules)

<img  src="images/cc_sa.png" width="200">